### Import require library first

In [1]:
from bs4 import BeautifulSoup
from html.parser import HTMLParser
import pandas as pd
from datetime import datetime
import os
import sys
import pickle
import pprint

import urllib.request, urllib.parse, urllib.error

proxies = {"http": "http://127.0.0.1:8090",
           "https": "http://127.0.0.1:8090"}

# http://squidman.net/squidman/
    
#create the object, assign it to a variable
proxy = urllib.request.ProxyHandler({'http': 'http://127.0.0.1:8090/','https': 'http://127.0.0.1:8090/'})
# construct a new opener using your proxy settings
opener = urllib.request.build_opener(proxy)
# install the openen on the module-level
urllib.request.install_opener(opener)

In [2]:
tmp_dict = {'HorseNo': None,
'HorseName': None,
'Age': None,
'JockeyName': None,
'ActualWt': None,
'Draw': None,
'Rate': None,
'Owner': None,
'DeclarHorseWt': None,
'WinOddBeforeNight': None,
'WinOddBeforeGate': None,             
'WinOdd': None,
'WinInTenThousand': None,
'PInTenThousand': None,
'P_Odd': None,
'PlaceInMiddle': None,
'Place': None,
'TimeInMiddle': None,
'FinishTime': None,
'LBW': None
}

In [3]:
def convert_str_to_int(int_str):
    try:
        return int(int_str)
    except Exception as err:
        return int_str

def convert_str_to_float(int_str):
    try:
        return float(int_str)
    except Exception as err:
        return int_str

In [4]:
def get(url, data_list=None, max_try=3):
    if data_list:
        url = "{}?{}".format(url, urllib.parse.urlencode(data_list))
    query = urllib.request.Request(url)
    current_try = 0
    while current_try < max_try:
        try:
            response = urllib.request.urlopen(query)
            html = response.read()
            response.close()
            return html
        except Exception as e:
            return None
    raise Exception("Cannot open page {}".format(url))

In [5]:
# url = 'http://www.hkhorsedb.com/cseh/passodds.php'
# html = get(url)
# soup = BeautifulSoup(str(html, 'big5'), "html.parser")

In [6]:
def get_race_date_array():
    url = 'http://www.hkhorsedb.com/cseh/passodds.php'
    html = get(url)
    soup = BeautifulSoup(str(html, 'big5'), "html.parser")
    tr_list = soup.findAll('table')[15].findAll('tr')[1:]
    
    race_date_array = []
    
    for i in range(0, len(tr_list)-1):
        tr = tr_list[i]
        tds = tr.findAll('td')
        
        for j in range(0, len(tds)-1):
            detail_date = tds[j].text
            
            if (detail_date != "-"):
                normal_data = ''.join(reversed(detail_date.split('-')))
                
                if(normal_data != ""):
                    race_date_array.append(normal_data)
    
    return race_date_array
    
race_date_array = get_race_date_array()
print(race_date_array[0:10])
print(race_date_array[-10:])

['20170621', '20170618', '20170614', '20170607', '20170604', '20170531', '20170524', '20170521', '20170517', '20170510']
['20051203', '20051127', '20051123', '20051120', '20051113', '20051109', '20051105', '20051030', '20051026', '20051022']


In [7]:
# url = 'http://www.hkhorsedb.com/cseh/passodds.php'
# html = get(url)
# soup = BeautifulSoup(str(html, 'big5'), "html.parser")

In [8]:
# tr_list = soup.findAll('table')[15].findAll('tr')[1:]

In [9]:
# print(tr_list[1])

In [10]:
# tr = tr_list[0]
# tds = tr.findAll('td')

In [11]:
# tds

In [12]:
# detail_date = tds[0].text
# detail_date

In [13]:
# date_info = ''.join(detail_date.split('-'))
# normal_data = ''.join(reversed(detail_date.split('-')))
# normal_data

In [14]:
# 'http://racing.hkjc.com/racing/Info/meeting/Results/english/Local/20130206/HV/5'

In [15]:
# tr_last = tr_list[-5]
# tds_last = tr_last.findAll('td')
# tds_last

In [16]:
def acquire_appledaily_race_record(date_str, race_id):
    url = 'http://hk.racing.nextmedia.com/fullresult.php?date={}&page={}'.format(date_str,race_id)
    html = get(url)
    soup = BeautifulSoup(str(html, 'utf-8'), "html.parser")
    record_trs = soup.findAll('table')[2].findAll('tr')[2:-1]
    return record_trs

In [17]:
# trs = acquire_appledaily_race_record("20130206","05")

In [18]:
# next_result = 'http://hk.racing.nextmedia.com/fullresult.php?date=20130206&page=05'

In [19]:
# html_next = get(next_result)

In [20]:
# soup_next = BeautifulSoup(str(html_next, 'utf-8'), "html.parser")

In [21]:
# next_tr_list = soup_next.findAll('table')[2].findAll('tr')[2:-1]
# next_tr_list[0]

In [22]:
# next_tr_list[-1]

In [23]:
# next_td_list = next_tr_list[0].findAll('td')

In [24]:
# next_td_list

In [25]:
# '馬號	馬名	歲	騎師	負磅	檔	評分	廄	馬匹體重	賠率	獨贏票(萬)	位置票(萬)	位置賠率	走位	名次	分段時間	總時間	勝負距離'

In [26]:
# '7	超新鮮	4	薛寶力	123	1	67	姚	1121	4.8	3.7	4.5	455	346	1.5	2 2	1	12.75 21.61 23.01	(0.57.37)	頭馬'

In [27]:
appledaily_web_columns = ['Date','RaceNumber','HorseNo','HorseName','Age','JockeyName','ActualWt', 'Draw', 'Rate','Owner','DeclarHorseWt','WinOddBeforeNight','WinOddBeforeGate','WinOdd','WinInTenThousand','PInTenThousand','P_Odd','PlaceInMiddle','Place', 'TimeInMiddle','FinishTime','LBW']

In [28]:
tmp_result = tmp_dict.copy()

In [29]:
save_path = '/Users/chris.chan/project/TradingProjects/HKHorseDB/data/race_records'

# setup the data array for later use
result_df = pd.DataFrame(columns=appledaily_web_columns)

def process_get_race_date_id(date_str, race_id): #, driver, race_course='ST'):
    
    filename = '{}_{}.xlsx'.format(date_str, race_id)
    
    failed_info = ''
    
    next_tr_list = acquire_appledaily_race_record(date_str,race_id)
    try:
        for index in range(0, len(next_tr_list)-1):
            tmp_result = tmp_dict.copy()

#             print(index)

            next_td_list = next_tr_list[index].findAll('td')
            next_td_list

            try:
                tmp_result['Date'] = date_str
            except: 
                print('Can not find Date')
            
            try:
                tmp_result['RaceNumber'] = race_id
            except: 
                print('Can not find Race Number')
            
            
            try:
                tmp_result['HorseNo'] = convert_str_to_int(next_td_list[0].text)
            except IndexError:
                print('Can not find HorseNo')

            try:
                tmp_result['HorseName'] = next_td_list[1].text
            except IndexError:
                print('Can not find HorseName')
                continue

            try:
                tmp_result['Age'] = convert_str_to_int(next_td_list[2].text)
            except IndexError:
                print('Can not find Age')

            try:
                tmp_result['JockeyName'] = next_td_list[3].text
            except IndexError:
                print('Can not find JockeyName')

            try:
                tmp_result['ActualWt'] = convert_str_to_int(next_td_list[4].text)
            except IndexError:
                print('Can not find ActualWt')

            try:
                tmp_result['Draw'] = convert_str_to_int(next_td_list[5].text)
            except IndexError:
                print('Can not find Draw')

            try:
                tmp_result['Rate'] = convert_str_to_int(next_td_list[6].text)
            except IndexError:
                print('Can not find Rate')

            try:
                tmp_result['Owner'] = next_td_list[7].text
            except IndexError:
                print('Can not find Owner')

            try:
                tmp_result['DeclarHorseWt'] = convert_str_to_int(next_td_list[8].text)
            except IndexError:
                print('Can not find DeclarHorseWt')

            try:
                tmp_result['WinOddBeforeNight'] = convert_str_to_float(next_td_list[9].text)
            except IndexError:
                print('Can not find WinOddBeforeNight')

            try:
                tmp_result['WinOddBeforeGate'] = convert_str_to_float(next_td_list[10].text)
            except IndexError:
                print('Can not find WinOddBeforeGate')

            try:
                tmp_result['WinOdd'] = convert_str_to_float(next_td_list[11].text)
            except IndexError:
                print('Can not find WinOdd')

            try:
                tmp_result['WinInTenThousand'] = convert_str_to_float(next_td_list[12].text)
            except IndexError:
                print('Can not find WinInTenThousand')

            try:
                tmp_result['PInTenThousand'] = convert_str_to_float(next_td_list[13].text)
            except IndexError:
                print('Can not find PInTenThousand')

            try:
                tmp_result['P_Odd'] = convert_str_to_float(next_td_list[14].text)
            except IndexError:
                print('Can not find P_Odd')

            try:
                tmp_result['PlaceInMiddle'] = next_td_list[15].text
            except IndexError:
                print('Can not find PlaceInMiddle')

            try:
                tmp_result['Place'] = convert_str_to_int(next_td_list[16].text)
            except IndexError:
                print('Can not find Place')

            try:
                tmp_result['TimeInMiddle'] = next_td_list[17].text
            except IndexError:
                print('Can not find TimeInMiddle')

            try:
                tmp_result['FinishTime'] = next_td_list[18].text
            except IndexError:
                print('Can not find FinishTime')

            try:
                tmp_result['LBW'] = next_td_list[19].text
            except IndexError:
                print('Can not find LBW')

            result_df.loc[index] = tmp_result
            
        result_df.to_excel(os.path.join(save_path, '{}_{}.xlsx'.format(date_str, race_id)), index=False)
        result_df.to_csv(os.path.join(save_path, '{}_{}.cvs'.format(date_str, race_id)), index=False)

    except Exception as err:
        failed_info = '{} {}: test_failed as {}'.format(failed_info, datetime.today().strftime('%Y-%m-%d'), err)
        print(failed_info)
        

In [30]:
process_get_race_date_id("20130206","05")

In [40]:
result_df

,Date,RaceNumber,HorseNo,HorseName,Age,JockeyName,ActualWt,Draw,Rate,Owner,...,WinOddBeforeGate,WinOdd,WinInTenThousand,PInTenThousand,P_Odd,PlaceInMiddle,Place,TimeInMiddle,FinishTime,LBW
0,20130206,05,7,超新鮮,4,薛寶力,123,1,67,姚,...,3.7,4.5,455.0,346.0,1.5,2 2,1,12.75 21.61 23.01,(0.57.37),頭馬
1,20130206,05,11,火燎腚,5,何澤堯,114,6,61,方,...,13.0,10.0,196.0,162.0,3.3,1 1,2,12.55 21.61 23.41,(0.57.57),1-1/4
2,20130206,05,10,星運俊爵,5,湯智傑,116,3,62,方,...,28.0,26.0,78.0,105.0,5.1,5 6,3,12.91 21.81 22.91,(0.57.63),1-1/2
3,20130206,05,8,晴空萬里,4,楊明綸,120,10,66,李,...,8.4,12.0,169.0,140.0,3.8,11 10,4,13.27 22.17 22.21,(0.57.65),1-3/4
4,20130206,05,6,百戰百勝,4,吳嘉晉,117,12,68,蔡,...,20.0,11.0,179.0,161.0,3.3,12 12,5,13.51 22.21 21.97,(0.57.69),2
5,20130206,05,1,捉金,5,鄭雨滇,133,9,77,伍,...,4.1,3.6,566.0,387.0,1.4,4 4,6,12.91 21.57 23.36,(0.57.84),3
6,20130206,05,4,四季星,3,蔡明紹,123,8,69,東,...,3.9,3.9,527.0,330.0,1.6,10 11,7,13.19 22.41 22.25,(0.57.85),3
7,20130206,05,5,太陽海岸,5,王志偉,122,5,68,葉,...,39.0,60.0,34.0,38.0,13.0,8 9,8,13.11 22.13 22.64,(0.57.88),3-1/4
8,20130206,05,3,奧妙繽紛,3,潘頓,125,7,69,方,...,17.0,27.0,76.0,73.0,7.4,7 7,9,13.03 21.93 22.94,(0.57.9),3-1/4
9,20130206,05,9,常山鳳寶,8,霍勵賢,120,4,64,告,...,41.0,35.0,57.0,67.0,8.1,3 3,10,12.75 21.65 23.63,(0.58.03),4-1/4


In [32]:
# Add Leading Zero
# str(1).zfill(2) 

In [33]:
# ~/Library/Logs/squid/squid-access.log

In [34]:
# hockey_web_columns = ['Place', 'HorseNo', 'HorseName', 'HorseCode', 'JockeyName', 'JockeyCode', 'TrainerName', 'TrianerCode',
#            'ActualWt', 'DeclarHorseWt', 'Draw', 'LBW', 'FinishTime', 'WinOdd', 'SecTime1', 'SecTime2', 'SecTime3',
#            'SecTime4', 'SecTime5', 'SecTime6']

In [35]:
# '名次	馬號	馬名	騎師	練馬師	實際負磅	排位體重	檔位	頭馬距離	沿途走位	完成時間	獨贏賠率'

In [36]:
# 'Plc.	Horse No.	Horse	Jockey	Trainer	Actual Wt.	Declar.Horse Wt.	Draw	LBW	Running Position	Finish Time	Win Odds'

In [37]:
# '1	7	SUPER FRESH(N194)	A Suborics	P F Yiu	123	1121	1	-	2	2	1	0.57.37	4.5'

In [38]:
# jackey_url = 'http://racing.hkjc.com/racing/Info/meeting/Results/english/Local/20130206/HV/5'

In [39]:
# 'http://bet.hkjc.com/racing/getXML.aspx?type=win&date=19-10-2016&venue=HV&raceno=1'